## Pruebas asociada al análisis de datos, comparativa FORvsUNWIND y consulta de recomendación basado en TAGs.

#### Comentarios previos:

- Instalacion de Neo4j 1.6.1 (pasos):
	- Descargar de https://neo4j.com/download/
	- Hecho lo anterior, en "Software registration" usar clave que da la página anterior.
	- Instalado el motor:
		- Clic en "Add" para crear un nuevo proyecto.
		- Dentro del proyecto, clic en "Add Graph" y seleccionar "Local DBMS"
		- Agregar name (ej: BDNR_ml-25m), pass: (ej: bdnr2025). En mi caso utilicé versión 5.24.0. Luego clic en "create"
		- Finalmente dar en "Start", sino Python no puede conectarse a localhost:7687 
        - En mi caso tuve algunos problemas de conexión, pero lo solución yendo a "Settings" (en los tres puntitos del proyecto), y descomentando las lineas: #server.bolt.listen_address=:7687 y #server.default_listen_address=0.0.0.0

- Descargamos datos de:
	- https://grouplens.org/datasets/movielens/25m/



A lo largo del notebook, realizamos el proceso completo para trabajar con una muestra representativa del dataset MovieLens 25M:
- Cargamos los datos y aplicamos un preprocesamiento cuidadoso para reducir volumen y mantener la coherencia entre entidades.
- Evaluamos el rendimiento entre dos técnicas comunes de carga en Neo4j: FOR + MERGE vs UNWIND + MERGE.
- Cargamos los distintos nodos y relaciones acorde al diseño propuesto (usando la estrategia más eficiente).
- Ejecutamos algunas consultas para verificar el contenido y evaluar el modelo.

#### Primeros pasos
Instalo librerías necesarias

In [13]:
#!pip install pandas 
#!pip install neo4j
#!pip install py2neo

Verifico si se instaló correctamanete e importo demás librerías a utilizar a lo largo del trabajo

In [26]:
import pandas as pd
import time  # Lo usamos para medir tiempos
from neo4j import GraphDatabase

print(neo4j.__version__)


2021.2.4


Importamos y nos contectamos a Neo4j:

In [28]:
from neo4j import GraphDatabase

uri = "bolt://localhost:7687"  
user = "neo4j"
password = "bdnr2025" 

driver = GraphDatabase.driver(uri, auth=(user, password))
session = driver.session()

try:
    session.run("RETURN 1").single()
    print("Successfully connected to Neo4j")
except Exception as e:
    print(f"Connection failed: {e}")

Successfully connected to Neo4j


#### Analisis del dataset:
- Leemos todos los archivos
- Creamos función para resumen rápido (calculo de cantidad de datos)

In [9]:
# Ruta base
base_path = "/Users/usuario/Desktop/BDNR/dataset/ml-25m"

# Leer datasets
ratings = pd.read_csv(f"{base_path}/ratings.csv")
movies = pd.read_csv(f"{base_path}/movies.csv")
tags = pd.read_csv(f"{base_path}/tags.csv")
genome_tags = pd.read_csv(f"{base_path}/genome-tags.csv")
genome_scores = pd.read_csv(f"{base_path}/genome-scores.csv")
links = pd.read_csv(f"{base_path}/links.csv")


In [10]:
def resumen_tablas_comparado(dfs_completos, dfs_filtrados, nombres):
    resumen = []
    for nombre, df_full, df_filt in zip(nombres, dfs_completos, dfs_filtrados):
        total_filas = df_full.shape[0]
        filtrado_filas = df_filt.shape[0]
        porcentaje_usado = round((filtrado_filas / total_filas) * 100, 2) if total_filas > 0 else 0
        resumen.append({
            "Archivo": nombre,
            "Filas totales": total_filas,
            "Filas filtradas": filtrado_filas,
            "% usado": f"{porcentaje_usado}%",
            "Nulos (total)": df_full.isnull().sum().sum(),
            "Nulos (filtrado)": df_filt.isnull().sum().sum(),
            "Tipos de datos": ", ".join(df_full.dtypes.astype(str).unique())
        })
    return pd.DataFrame(resumen)


#### Carga de datos y preprocesamiento
Debido al tamaño del dataset completo (más de 25 millones de ratings), trabajamos con una muestra representativa que incluye:
- 2025 usuarios únicos seleccionados aleatoriamente.
- Las películas asociadas a esos usuarios.
- Todos los ratings, tags y scores vinculados a esas películas.
- Todas las etiquetas (genome_tags) ya que son pocas (1128).
- En el caso de las películas, procesamos el campo genres (cadena separada por "|") para convertirlo en una lista de géneros. Además, calculamos otros valores estadísticos a utilizar en el sistema de recomendación.


In [15]:
# Filter Data
SAMPLE_SIZE = 2025

selected_users = ratings['userId'].unique()[:SAMPLE_SIZE]
filtered_ratings = ratings[ratings['userId'].isin(selected_users)]
filtered_movie_ids = filtered_ratings['movieId'].unique()
filtered_movies = movies[movies['movieId'].isin(filtered_movie_ids)].copy()
filtered_tags = tags[tags['movieId'].isin(filtered_movie_ids)]
filtered_scores = genome_scores[genome_scores['movieId'].isin(filtered_movie_ids)]


Resumen de los datos filtrados

In [16]:
dfs_completos = [ratings, movies, tags, genome_tags, genome_scores]
dfs_filtrados = [filtered_ratings, filtered_movies, filtered_tags, genome_tags, filtered_scores]
nombres = ["ratings", "movies", "tags", "genome_tags", "genome_scores"]

resumen_tablas_comparado(dfs_completos, dfs_filtrados, nombres)



,Archivo,Filas totales,Filas filtradas,% usado,Nulos (total),Nulos (filtrado),Tipos de datos
0,ratings,25000095,293308,1.17%,0,0,"int64, float64"
1,movies,62423,14619,23.42%,0,0,"int64, object"
2,tags,1093360,924902,84.59%,16,16,"int64, object"
3,genome_tags,1128,1128,100.0%,0,0,"int64, object"
4,genome_scores,15584448,13044192,83.7%,0,0,"int64, float64"


Observación: El campo `timestamp` en `ratings` representa una marca temporal expresada como número entero (segundos desde el 1/1/1970 UTC). Más adelante evaluaremos su transformación y uso para consultas temporales.

#### Evaluación de rendmiento: FOR + MERGE vs UNWIND + MERGE
En este experimento, buscamos comparar dos formas comunes de insertar datos en Neo4j:
- UNWIND + MERGE: estrategia recomendada, basada en procesamiento por lotes y más eficiente para grandes volúmenes.
- FOR + MERGE: estrategia tradicional, menos eficiente al realizar una transacción por iteración.

Ambas pruebas se realizan sobre la misma submuestra de ratings, comparando los tiempos de ejecución.

In [17]:
# Subconjunto de datos
subset_ratings = filtered_ratings.head(1000)
rating_data = subset_ratings.to_dict("records")

# 1a. Ratings -> nodos User, Movie y relaciones RATED (Estrategia UNWIND + MERGE)
start_time = time.time()
graph.run("""
UNWIND $data AS row
MERGE (u:User {userId: row.userId})
MERGE (m:Movie {movieId: row.movieId})
MERGE (u)-[r:RATED]->(m)
SET r.rating = row.rating, r.timestamp = row.timestamp
""", data=rating_data)
end_time = time.time()
print(f"Tiempo usando UNWIND + MERGE: {end_time - start_time:.2f} segundos")

# 1b. Ratings -> nodos User, Movie y relaciones RATED (Estrategia FOR + MERGE)
graph.run("MATCH (n) DETACH DELETE n")  # Limpiamos antes de volver a cargar
start_time = time.time()
for _, row in subset_ratings.iterrows():
    graph.run("""
        MERGE (u:User {userId: $userId})
        MERGE (m:Movie {movieId: $movieId})
        MERGE (u)-[r:RATED]->(m)
        SET r.rating = $rating, r.timestamp = $timestamp
    """, userId=int(row["userId"]),
         movieId=int(row["movieId"]),
         rating=float(row["rating"]),
         timestamp=int(row["timestamp"]))
end_time = time.time()
print(f"Tiempo usando FOR + MERGE: {end_time - start_time:.2f} segundos")


Tiempo usando UNWIND + MERGE: 5.60 segundos
Tiempo usando FOR + MERGE: 54.28 segundos


Como era de esperarse, UNWIND demostró ser mucho más eficiente al realizar la operación en bloque. Esta técnica será la utilizada en el resto de la carga para mantener la eficiencia.

#### Cargar nodos y relaciones:
Usamos modelo detallado en /notebooks/benchmark/neo4j.ipynb

In [19]:
from collections import defaultdict
# Create a dictionary of genome tags for quick lookup
genome_tags_dict = {row['tagId']: row['tag'] for _, row in genome_tags.iterrows()}

# Build dictionaries to accumulate data
ratings_by_movie = defaultdict(list)
scores_by_movie = defaultdict(list)

ratings_by_user = defaultdict(list)
tags_by_user = defaultdict(list)

In [20]:
# Process ratings and tags to build both movie and user views
RELEVANCE_THRESHOLD = 0.8

# Ratings
for _, r in filtered_ratings.iterrows():
    movie_id = int(r.movieId)
    user_id = int(r.userId)
    
    # Add to movie's ratings
    ratings_by_movie[movie_id].append({
        "userId": user_id,
        "rating": float(r.rating),
        "timestamp": int(r.timestamp)
    })
    
    # Add to user's ratings
    ratings_by_user[user_id].append({
        "movieId": movie_id,
        "rating": float(r.rating),
        "timestamp": int(r.timestamp)
    })
    
# Tags (only for users), genome tag is the cleaned infered tag
for _, t in filtered_tags.dropna(subset=["tag"]).iterrows():
    movie_id = int(t.movieId)
    user_id = int(t.userId)
    
    tags_by_user[user_id].append({
        "movieId": movie_id,
        "tag": t.tag,
        "timestamp": int(t.timestamp)
    })
    
# Genome scores (only for movies), with 0.8 threshold
for _, s in filtered_scores[filtered_scores.relevance > RELEVANCE_THRESHOLD].iterrows():
    movie_id = int(s.movieId)
    tag_id = int(s.tagId)
    
    scores_by_movie[movie_id].append({
        "tagId": tag_id,
        "tag": genome_tags_dict.get(tag_id, ""),
        "relevance": float(s.relevance)
    })

In [21]:
import re

# Build movie documents
movie_docs = []
for _, m in filtered_movies.iterrows():
    movie_id = int(m.movieId)
   
    # Extract year from title
    title = m.title
    year_match = re.search(r"\((\d{4})\)$", title)
    year = int(year_match.group(1)) if year_match else None
    
    # Get ratings and tags for this movie
    movie_ratings = ratings_by_movie.get(movie_id, [])
    tag_entries = scores_by_movie.get(movie_id, [])
    
    # Calculate statistics
    rating_count = len(movie_ratings)
    if rating_count > 0:
        avg_rating = sum(r['rating'] for r in movie_ratings) / rating_count
        # Calculate rating distribution (count of each star rating)
        rating_dist = {"0.5":0, "1.0":0, "1.5":0, "2.0":0, "2.5":0, 
                      "3.0":0, "3.5":0, "4.0":0, "4.5":0, "5.0":0}
        for r in movie_ratings:
            rating_key = f"{r['rating']}"
            rating_dist[rating_key] += 1 
    else:
        avg_rating = 0.0
        rating_dist = {}  # will result in empty arrays below

    # Neo4j only allows:
    # - Single values: string, int, float, boolean, datetime
    # - Arrays of primitive types: [int], [string], etc.
    dist_keys = list(rating_dist.keys()) if rating_dist else []
    dist_values = list(rating_dist.values()) if rating_dist else []
    
    tag_ids = [int(e["tagId"]) for e in tag_entries]
    tag_relevances = [float(e["relevance"]) for e in tag_entries]
    tag_names = [str(e["tag"]) for e in tag_entries]
    
    movie_docs.append({
        "movieId": movie_id,
        "title": title,
        "year": year,
        "genres": m.genres.split("|") if pd.notna(m.genres) else [],
        "ratingCount": rating_count,
        "avgRating": avg_rating,
        "ratingDistKeys": dist_keys,
        "ratingDistValues": dist_values,
        "tagIds": tag_ids,
        "tagRelevances": tag_relevances,
        "tagNames": tag_names
    })

In [22]:
user_docs = []

for user_id in selected_users:
    user_id = int(user_id)

    user_ratings = ratings_by_user.get(user_id, [])
    user_tags = tags_by_user.get(user_id, [])
    
    # Rating stats
    rating_count = len(user_ratings)
    avg_rating = sum(r['rating'] for r in user_ratings) / rating_count if rating_count > 0 else 0.0
    
    # Sort ratings by timestamp
    user_ratings.sort(key=lambda r: r['timestamp'])

    # Flatten ratings for Neo4j
    rated_movie_ids = [r['movieId'] for r in user_ratings]
    rated_scores = [r['rating'] for r in user_ratings]
    rated_timestamps = [r['timestamp'] for r in user_ratings]

    # (Optional future support for tags)
    # tag_movie_ids = [t['movieId'] for t in user_tags]
    # tag_labels = [t['tag'] for t in user_tags]
    # tag_timestamps = [t['timestamp'] for t in user_tags]

    user_docs.append({
        "userId": user_id,
        "ratingCount": rating_count,
        "avgRating": round(avg_rating, 2),
        "ratedMovieIds": rated_movie_ids,
        "ratedScores": rated_scores,
        "ratedTimestamps": rated_timestamps,
        # "tagMovieIds": tag_movie_ids,
        # "tagLabels": tag_labels,
        # "tagTimestamps": tag_timestamps
    })

In [23]:
from neo4j import Session

def load_users(session: Session, user_data: list, batch_size: int = 1000) -> None:
    """Creates User nodes with rating history embedded as arrays."""
    
    query = """
    UNWIND $batch AS user
    MERGE (u:User {userId: user.userId})
    SET u.ratingCount = user.ratingCount,
        u.avgRating = user.avgRating,
        u.ratedMovieIds = user.ratedMovieIds,
        u.ratedScores = user.ratedScores,
        u.ratedTimestamps = user.ratedTimestamps
    """

    if batch_size is None:
        batch_size = len(user_data)

    for i in range(0, len(user_data), batch_size):
        batch = user_data[i:i + batch_size]
        session.run(query, batch=batch)


def load_movies(session: Session, movie_data: list, batch_size: int = 1000) -> None:
    """Creates Movie nodes with embedded genome tags (tagGenome)."""
         
    # Cypher query to store Movie nodes with embedded data
    query = """
    UNWIND $batch AS movie
    MERGE (m:Movie {movieId: movie.movieId})
    SET m.title = movie.title,
        m.year = movie.year,
        m.genres = movie.genres,
        m.ratingCount = movie.ratingCount,
        m.avgRating = movie.avgRating,
        m.ratingDistKeys = movie.ratingDistKeys,
        m.ratingDistValues = movie.ratingDistValues,        
        m.tagIds = movie.tagIds,
        m.tagRelevances = movie.tagRelevances,
        m.tagNames = movie.tagNames
    """
    
    if batch_size is None:
        batch_size = len(movie_data)

    for i in range(0, len(movie_data), batch_size):
        batch = movie_data[i:i + batch_size]
        session.run(query, batch=batch)


def load_ratings(session: Session, ratings_df: pd.DataFrame, batch_size: int = 1000) -> None:
    """Creates User and Movie nodes with RATED relationships."""
    rating_data = ratings_df.to_dict("records")
    
    query = """
    UNWIND $batch AS row
    MERGE (u:User {userId: row.userId})
    MERGE (m:Movie {movieId: row.movieId})
    MERGE (u)-[r:RATED]->(m)
    SET r.rating = row.rating, 
        r.timestamp = row.timestamp
    """
   
    if batch_size is None:
        batch_size = len(ratings_df)
    
    # Process in batches to avoid memory issues
    for i in range(0, len(rating_data), batch_size):
        batch = rating_data[i:i + batch_size]
        session.run(query, batch=batch)

def load_tags(session: Session, tags_df: pd.DataFrame, batch_size: int = 1000) -> None:
    """Creates TAGGED relationships between existing Users and Movies"""
    # Clean data - remove null tags
    tag_data = tags_df.dropna(subset=["tag"]).to_dict("records")
    
    query = """
    UNWIND $batch AS row
    MATCH (u:User {userId: row.userId})
    MATCH (m:Movie {movieId: row.movieId})
    MERGE (u)-[t:TAGGED]->(m)
    SET t.tag = row.tag
    """
       
    if batch_size is None:
        batch_size = len(tags_df)
        
    for i in range(0, len(tag_data), batch_size):
        batch = tag_data[i:i + batch_size]
        session.run(query, batch=batch)

In [30]:
result = session.run("MATCH (n) DETACH DELETE n")
summary = result.consume()

print("Query summary:")
print(f"Nodes deleted: {summary.counters.nodes_deleted}")
print(f"Relationships deleted: {summary.counters.relationships_deleted}")

Query summary:
Nodes deleted: 0
Relationships deleted: 0


In [31]:
import time

start_time = time.time()

stats = {
    'movies_loaded': 0,
    'users_loaded': 0,
    'rates_created': 0,
    'tags_created': 0,
    'execution_time': 0
}

BATCH_SIZE = None

# Insert data
try:
    print("Starting data loading process...")
    
    # Load users (foundational nodes)
    print("\nLoading users...")
    user_start = time.time()
    load_users(session, user_docs, BATCH_SIZE)
    stats['users_loaded'] = len(user_docs)
    print(f"Users loaded in {time.time() - user_start:.2f}s")
    
    # Load movies (foundational nodes)
    print("\nLoading movies...")
    movie_start = time.time()
    load_movies(session, movie_docs, BATCH_SIZE)
    stats['movies_loaded'] = len(movie_docs)
    print(f"Movies loaded in {time.time() - movie_start:.2f}s")
    
    # Load users through ratings
    print("\nLoading ratings...")
    ratings_start = time.time()
    load_ratings(session, filtered_ratings, BATCH_SIZE)
    stats['rates_created'] = len(filtered_ratings)
    print(f"Ratings loaded in {time.time() - ratings_start:.2f}s")
    
    # Load tags
    print("\nLoading tags...")
    tags_start = time.time()
    load_tags(session, filtered_tags, BATCH_SIZE)
    stats['tags_created'] = len(filtered_tags)
    print(f"Tags loaded in {time.time() - tags_start:.2f}s")

except Exception as e:
    print(f"\nError during data loading: {str(e)}")
    raise
finally:
    stats['execution_time'] = time.time() - start_time
    print(f"\nTotal processing time: {stats['execution_time']:.2f} seconds")
    print(f"Summary: {stats}")

Starting data loading process...

Loading users...
Users loaded in 7.12s

Loading movies...
Movies loaded in 83.67s

Loading ratings...
Ratings loaded in 1535.25s

Loading tags...
Tags loaded in 584.10s

Total processing time: 2210.14 seconds
Summary: {'movies_loaded': 14619, 'users_loaded': 2025, 'rates_created': 293308, 'tags_created': 924902, 'execution_time': 2210.138160943985}


#### Consulta sobre recomendación:

- Devuelve películas similares en etiquetas relevantes (score > 0.8).
- Muestra el número de etiquetas en común (cantidad_etiquetas), promedio de calificación (avgRating) y cantidad de calificaciones (ratingCount).
- Ordena por: cantidad_etiquetas (desc), avgRating (desc), ratingCount (desc).

In [52]:
def peliculas_similares_embebidas(title, top):
    query = """
    MATCH (m1:Movie {title: $title})
    WITH m1,
         [i IN range(0, size(m1.tagNames)-1)
          WHERE m1.tagRelevances[i] > 0.8 | m1.tagIds[i]] AS etiquetas_relevantes

    MATCH (m2:Movie)
    WHERE m2 <> m1

    WITH m1, m2, etiquetas_relevantes,
         [i IN range(0, size(m2.tagNames)-1)
          WHERE m2.tagRelevances[i] > 0.8 | m2.tagIds[i]] AS etiquetas_m2

    WITH m2.title AS pelicula_similar,
         m2.avgRating AS avgRating,
         m2.ratingCount AS ratingCount,
         REDUCE(s = 0, x IN etiquetas_relevantes |
            s + CASE WHEN x IN etiquetas_m2 THEN 1 ELSE 0 END
         ) AS cantidad_etiquetas

    WHERE cantidad_etiquetas > 0

    RETURN pelicula_similar, cantidad_etiquetas, avgRating, ratingCount
    ORDER BY cantidad_etiquetas DESC, avgRating DESC, ratingCount DESC
    LIMIT $top
    """
    return graph.run(query, title=title, top=top).to_table()


In [55]:
nombre_peliculas = ["Toy Story (1995)", "Matrix, The (1999)", "Forrest Gump (1994)"]
for pelicula in nombre_peliculas:
    print(f"Películas similares a \"{pelicula}\" (basado en etiquetas)\n")
    start = time.time()
    resultados = peliculas_similares_embebidas(title=pelicula, top=10)
    print(resultados)
    end = time.time()
    print(f"Tiempo de ejecución: {(end - start)*1000:.2f} ms\n --------------------- \n")


Películas similares a "Toy Story (1995)" (basado en etiquetas)

 pelicula_similar                | cantidad_etiquetas |          avgRating | ratingCount 
---------------------------------|--------------------|--------------------|-------------
 Finding Nemo (2003)             |                 24 |  3.808235294117647 |         425 
 Monsters, Inc. (2001)           |                 23 | 3.8531553398058254 |         412 
 Toy Story 2 (1999)              |                 23 |  3.803174603174603 |         315 
 Bug's Life, A (1998)            |                 19 |  3.507042253521127 |         284 
 Up (2009)                       |                 17 | 3.9839285714285713 |         280 
 How to Train Your Dragon (2010) |                 17 |  3.914110429447853 |         163 
 Toy Story 3 (2010)              |                 17 |  3.861271676300578 |         173 
 Incredibles, The (2004)         |                 17 | 3.8453333333333335 |         375 
 Ratatouille (2007)              |  